# Tut4 - Using Access Token Authentication and Updating Domo via API
In an ideal world Domo 'wants' you use Access Token authentication.

Can we
1. monitor network traffic to see how an access token gets issued
2. capture that access token and upload it to an abstract account object in DomoJupyter?

3. 🚧 for take home extra credit
    - Using Tutorial 3 - create a dataset that lists available access tokens
    - Using Tutorial 2 - create a function that revokes an access token

RECALL:
- client_id and secret auth (we will call developer_token authentication moving forward) only applies to a small handful of APIs documented under developer.domo.com

- "full authentication actually has two flavors.<br>
    - username and password authentication (session_token which has a short expiration date)
    - access_token authentication (Admin > Security > Access Token) which tends to have a longer expiration date, we will cover this later
    
DEVELOPER TIP
- to have recycleable code, consider moving functions into a separate .py file in a subfolder.
- notice you must have an `__ini__.py` file in the subfolder in order to import it as a module

# Auth

In [1]:
import functions.utils as utils
import functions.auth as auth
import functions.errors as errors

In [2]:
creds = utils.get_account_credentials("username_password_auth", is_abstract_account=True)
assert creds

domo_instance = creds['DOMO_INSTANCE']

In [3]:
session_token = auth.get_full_auth(domo_username = creds['DOMO_USERNAME'],
                              domo_password = creds['DOMO_PASSWORD'],
                              domo_instance = creds['DOMO_INSTANCE']
                            )

assert session_token

In [4]:
import requests

# ultimately we will move this to auth.py, but surfacing here since this is a new function for us

def who_am_i(domo_instance, session_token):
    """uses session_token and returns who API request is authorized as"""
    
    headers = {"x-domo-authentication": session_token}
    url = f"https://{domo_instance}.domo.com/api/content/v2/users/me"
    
    res = requests.request(method = 'GET', url = url, headers = headers)
    
    data = res.json()
    
    if not res.ok:
        raise errors.DomoAPIRequest_Error(data)
    return data

who_am_i(domo_instance = domo_instance, session_token = session_token)

{'id': 1893952720,
 'invitorUserId': 587894148,
 'displayName': 'Jae Wilson1',
 'department': 'Business Improvement',
 'userName': 'jae@onyxreporting.com',
 'emailAddress': 'jae@onyxreporting.com',
 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66',
 'accepted': True,
 'userType': 'USER',
 'modified': 1708035568417,
 'created': 1588960518,
 'active': True,
 'pending': False,
 'anonymous': False,
 'systemUser': False}

## utils

In [5]:
import datetime as dt
import time
def generate_expiration_unixtimestamp(
    duration_in_days: int = 30, debug_prn: bool = False
):
    """generate a unixtimestamp `duration_in_days` into the future"""

    today = dt.datetime.today()
    expiration_date = today + dt.timedelta(days=duration_in_days)

    if debug_prn:
        print(f"expiration_date is {duration_in_days} from today {expiration_date}")

    return int(time.mktime(expiration_date.timetuple()) * 1000)

print(f"{dt.datetime.utcfromtimestamp(generate_expiration_unixtimestamp(5)/1000)} is 5 days in the future")

2024-02-28 02:50:32 is 5 days in the future


# Access Token


In [6]:
import datetime as dt

def generate_request_accesstoken_body(token_name, owner_id, duration_in_days: int = 30, debug_prn: bool = False):
    
    expires_ts = generate_expiration_unixtimestamp(duration_in_days)
    
    res = {"name":token_name,"ownerId":owner_id,"expires":expires_ts}
    
    if debug_prn:
        print(f"🚀 generating token for {owner_id} - will expire on {dt.datetime.utcfromtimestamp(expires_ts/1000)}")
    
    return res

generate_request_accesstoken_body(token_name = 'test', owner_id = 123, debug_prn = True)

🚀 generating token for 123 - will expire on 2024-03-24 02:50:32


{'name': 'test', 'ownerId': 123, 'expires': 1711248632000}

In [12]:
def request_access_token(
    token_name : str,
    session_token, 
    domo_instance,
    duration_in_days : int= 30,
    owner_id = None, # if no user_id provided, will generate for authenticated user
    debug_prn:bool = False
):
    
    # update user_id based on who_am_i
    res = who_am_i(domo_instance = domo_instance, 
                   session_token = session_token)
    
    owner_id = owner_id or res['id']
    
    body = generate_request_accesstoken_body(token_name,
                                             owner_id = owner_id,
                                             duration_in_days = duration_in_days,
                                             debug_prn = debug_prn)
    
    headers = {"x-domo-authentication": session_token}
    
    url = f'https://{domo_instance}.domo.com/api/data/v1/accesstokens'
    
    res = requests.request(method = 'POST', 
                           url = url, 
                           json = body,
                           headers = headers)
    
    data = res.json()
    
    if not res.ok:
        raise errors.DomoAPIRequest_Error(data)
        
    return data

access_token = request_access_token(token_name = "tutorial", 
                                    session_token= session_token,
                                    domo_instance = domo_instance
                                   )
access_token

{'id': 186721,
 'name': 'tutorial',
 'ownerId': 1893952720,
 'ownerName': 'Jae Wilson1',
 'ownerEmail': 'jae@onyxreporting.com',
 'expires': 1711249039000,
 'token': '51f7ee083ff2306230dbdb780ba71435b57c6f5ae498709f'}

## use an Abstract Account to store credentials

Remember, once we retrieve the token, we'll never see it again in clear text

How might we store it in an abstract account object?

Remember in tutorial 3 we generated a DomoStats dataset.  
1. retrieve the desired data
2. structure the data the way we want it
3. put it someplace

At this point we have retrieved the data, now we just need to understand how the DomoAccounts api needs to receive the access token for storage

1. monitor network traffic to see what it takes to store an abstract credential.

Homework
We don't always want to create a new account object, when we revoke and generate a new access_token, we probably just want to update the existing account object.  How might we.

1. get_all_accounts (already done!)
2. retrieve "the correct" account object (search by name)
3. update "the correct" account object

In [20]:
import datetime as dt
import json
def generate_abstract_credential_account_body( account_name, credentials, is_timestamp: bool = True):
    
    # the API expects credentials to be type string, so conditionally convert dict to string
    if isinstance(credentials, dict):
        credentials = json.dumps(credentials)
    
    if is_timestamp:
        account_name = f"{account_name} - updated {dt.date.today()}"
    return {"name":"Abstract Credential Store Account",
            "displayName": account_name,
            "dataProviderType":"abstract-credential-store",
            "configurations":{"credentials": credentials}}

generate_abstract_credential_account_body( "test", "my creds go here", is_timestamp = True)


{'name': 'Abstract Credential Store Account',
 'displayName': 'test - updated 2024-02-23',
 'dataProviderType': 'abstract-credential-store',
 'configurations': {'credentials': 'my creds go here'}}

In [22]:
import requests
def create_account(body, domo_instance, session_token):
    url = f"https://{domo_instance}.domo.com/api/data/v1/accounts"
    
    headers = {"x-domo-authentication": session_token}
    
    res = requests.request(method = 'POST', url = url, json = body, headers = headers)
    data = res.json()
    
    if not res.ok:
        raise errors.DomoAPIRequest_Error(data)
        
    return data

create_account_body = generate_abstract_credential_account_body(account_name = "my_domo_community_access_token", credentials = access_token)

create_account(body = create_account_body,
               domo_instance = domo_instance,
               session_token = session_token)

{'id': 94,
 'userId': 1893952720,
 'name': 'Abstract Credential Store Account',
 'displayName': 'my_domo_community_access_token - updated 2024-02-23',
 'type': 'data',
 'valid': False,
 'dataProviderType': 'abstract-credential-store',
 'credentialsType': 'fields',
 'createdAt': None,
 'createdBy': 1893952720,
 'modifiedAt': None,
 'modifiedBy': 1893952720,
 'configurations': {},
 'accountId': 94,
 'accountTemplateId': None,
 'accountTemplateAuthorizationId': None}

In [23]:

def main():
    
    creds = utils.get_account_credentials("username_password_auth", is_abstract_account=True)
    domo_instance = creds['DOMO_INSTANCE']
    
    session_token = auth.get_full_auth(domo_username = creds['DOMO_USERNAME'],
                              domo_password = creds['DOMO_PASSWORD'],
                              domo_instance = creds['DOMO_INSTANCE']
                            )
    
    access_token = request_access_token(token_name = "b", 
                                    session_token= session_token,
                                    domo_instance = domo_instance
                                   )
    
    create_account_body = generate_abstract_credential_account_body(account_name = "my_domo_community_access_token", credentials = access_token)

    return create_account(body = create_account_body,
               domo_instance = domo_instance,
               session_token = session_token)

main()

{'id': 95,
 'userId': 1893952720,
 'name': 'my_domo_community_access_token - updated 2024-02-23 (2)',
 'displayName': 'my_domo_community_access_token - updated 2024-02-23 (2)',
 'type': 'data',
 'valid': False,
 'dataProviderType': 'abstract-credential-store',
 'credentialsType': 'fields',
 'createdAt': None,
 'createdBy': 1893952720,
 'modifiedAt': None,
 'modifiedBy': 1893952720,
 'configurations': {},
 'accountId': 95,
 'accountTemplateId': None,
 'accountTemplateAuthorizationId': None}